In [255]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyxirr import xirr
pd.options.display.float_format = '{:,.2f}'.format


# Imputs

In [256]:
df_input=pd.read_csv('./input_fluxo_caixa.csv',header=None,index_col=0)
df_input=df_input.rename(columns={1:'valor_variavel'})

In [257]:
valor_imovel=df_input.loc['valor_imovel','valor_variavel']
entrada_pp=df_input.loc['entrada_pp','valor_variavel']
n_parcelas=int(df_input.loc['n_parcelas','valor_variavel'])                      # 35 anos
juros_nominal=df_input.loc['juros_nominal','valor_variavel']                # a.a

# Custos Iniciais

In [258]:
valor_entrada=valor_imovel*entrada_pp

custo_itbi=0.03*valor_imovel
custo_escritura=df_input.loc['custo_escritura','valor_variavel']
custo_registro=df_input.loc['custo_registro','valor_variavel']
custo_avaliacao=df_input.loc['custo_avaliacao','valor_variavel']

total_custos_iniciais=valor_entrada+custo_itbi+custo_escritura+custo_registro+custo_avaliacao


# Cálculo financiamento

In [259]:
juros_efetivo=((1+juros_nominal/12)**(12)-1)            # Efeito anual dos juros sobre juros mensais.  

valor_finaciado=valor_imovel-valor_entrada

encargos_finaciamento=df_input.loc['encargos_finaciamento','valor_variavel']

In [260]:
amortizacao=valor_finaciado/n_parcelas
saldo_devedor=valor_finaciado

# Cálculo do juros em função do mês
def parcela(n):
    saldo_devedor=valor_finaciado-(n-1)*amortizacao
    juros=saldo_devedor*juros_nominal/12
    return amortizacao+juros+encargos_finaciamento

df=pd.DataFrame(columns=['parcela','saldo_devedor'],index=range(0,n_parcelas))


for i in range(0,n_parcelas+1):
    if i==0:
        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=0
    else:

        df.loc[i,'saldo_devedor']=valor_finaciado-i*amortizacao
        df.loc[i,'parcela']=parcela(i)

# Custos fixos

In [261]:
condominio=df_input.loc['condominio','valor_variavel']
iptu_am=df_input.loc['iptu_aa','valor_variavel']/12
luz=df_input.loc['luz','valor_variavel']

total_custos_fixos=condominio+iptu_am+luz

In [262]:
for i in range(0,len(df)):
    df.loc[i,'custos_fixos']=total_custos_fixos
    if i==0:
        df.loc[i,'custos_iniciais']=total_custos_iniciais
    else:
        df.loc[i,'custos_iniciais']=0

# Custos com reforma

In [263]:
obra=df_input.loc['obra','valor_variavel']
prazo_obra=df_input.loc['prazo_obra','valor_variavel']                # Prazo da obra em mesês

def reforma(i, n,total):
    if i==0:
        return 0
    elif i<=n:
        return total/n
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'reforma']=reforma(i,prazo_obra,obra)

# Cenários de venda

In [264]:
valor_venda=df_input.loc['valor_venda','valor_variavel']
prazo_venda=int(prazo_obra+df_input.loc['prazo_venda_pos_obra','valor_variavel'])                # Prazo da obra em mesês + venda
comissao=df_input.loc['comissao','valor_variavel']

In [265]:
def venda(i, n,total):
    if i==n:
        print(f'Comissão: {comissao*valor_venda}')
        return -total*(1-comissao)
    else:
        return 0

for i in range(0,len(df)):
    df.loc[i,'venda']=venda(i,prazo_venda,valor_venda)
df['total']=df.sum(axis=1)
df['total']=df['total']-df['saldo_devedor']

# quitação do saldo devedor e anulação das despesas à partir da venda
for i in range(0,len(df)):
    if i>prazo_venda:
        df.loc[i,'total']=0
    elif i==prazo_venda:
        df.loc[i,'total']=df.loc[i,'total']+df.loc[i,'saldo_devedor']

Comissão: 22725.0


In [266]:
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
start_date = datetime.today().date()+15*delta_d

for i in range(0, len(df)):
    if i==0:
        df.loc[i,'data']=start_date
    else:
        df.loc[i,'data']=start_date+i*delta

df.head(12)

,parcela,saldo_devedor,custos_fixos,custos_iniciais,reforma,venda,total,data
0,0,"576,000.00",951.50,"174,600.00",0.00,0.00,"175,551.50",2022-08-15
1,"5,336.63","574,628.57",951.50,0.00,"23,750.00",0.00,"30,038.13",2022-09-15
2,"5,327.50","573,257.14",951.50,0.00,"23,750.00",0.00,"30,029.00",2022-10-15
3,"5,318.37","571,885.71",951.50,0.00,"23,750.00",0.00,"30,019.87",2022-11-15
4,"5,309.23","570,514.29",951.50,0.00,"23,750.00",0.00,"30,010.73",2022-12-15
5,"5,300.10","569,142.86",951.50,0.00,0.00,0.00,"6,251.60",2023-01-15
6,"5,290.97","567,771.43",951.50,0.00,0.00,"-886,275.00","-312,261.10",2023-02-15
7,"5,281.84","566,400.00",951.50,0.00,0.00,0.00,0,2023-03-15
8,"5,272.71","565,028.57",951.50,0.00,0.00,0.00,0,2023-04-15
9,"5,263.58","563,657.14",951.50,0.00,0.00,0.00,0,2023-05-15


In [267]:
# calculo da IRR
def cal_irr(x):
    
    dates=list(x['data'])
    amounts=list(x['total'])
    t_aa=xirr(dates, amounts)
    t_am=(1+t_aa)**(1/12)-1
    print('TII a.a.(%): ',t_aa*100)
    print('TII a.m.(%): ',t_am*100)
    print('Soma dos valores: ',-x['total'].sum())
    # return (t_aa*100,t_am*100,-x['total'].sum())

In [268]:
cal_irr(df)

TII a.a.(%):  8.518743977172852
TII a.m.(%):  0.6835986718487952
Soma dos valores:  10360.271428571432


In [269]:
### projeções de inflação e crescimento econômico para evolução dos preços de venda